In [3]:
import getpass
import os
import torch

#os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
torch.cuda.empty_cache()

In [5]:
torch.cuda.is_available()

False

In [2]:
# Load pdf

from langchain.document_loaders import PyMuPDFLoader

file_path = "C:/Users/Asus/Downloads/Informe_Tecnico_InstrumentoFortalecePYME.pdf"

loader = PyMuPDFLoader(file_path=file_path)

docs = loader.load()

print(len(docs))

36


In [4]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

1 
 
 
       
 
 
 
 
 
 
 
 
UNIVERSIDAD CATÓLICA DEL NORTE 
Facultad de Economía y Administración 
Centro de Investigación en Gestión de Tecnologías para la Empresa 
 
 
 
 
Informe Técnico  
 
“In

{'source': 'C:/Users/Asus/Downloads/Informe_Tecnico_InstrumentoFortalecePYME.pdf', 'file_path': 'C:/Users/Asus/Downloads/Informe_Tecnico_InstrumentoFortalecePYME.pdf', 'page': 0, 'total_pages': 36, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Versión 14.4.1 (Compilación 23E224) Quartz PDFContext', 'creationDate': "D:20240531180910Z00'00'", 'modDate': "D:20240531180910Z00'00'", 'trapped': ''}


In [3]:
# Make splits

from langchain_text_splitters import  RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

177


In [4]:
# Generate embeddings

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-V2")

c:\segment_anything\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Vector store

from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(documents=all_splits, embedding=embeddings)

ids = vector_store.add_documents(documents=all_splits)

In [8]:
# Example for text quering with similarity search

results = vector_store.similarity_search_with_score(
    "¿Se mide el nivel de madurez de las empresas?"
)

doc, score = results[0]
print(f"Score: {score}")
print(doc)

Score: 0.6929181814193726
page_content='en base a los puntajes obtenidos y las ponderaciones de cada dimensión se procederá a 
definir la madurez de cada empresa.  
 
Tabla 3. Ponderaciones nivel de madurez digital. 
 
Nivel de madurez digital 
Nivel 0: Análogo/Manual (1 a 1,8) 
Nivel 1: Iniciado (1,9 a 2,6) 
Nivel 2: Intermedio (2,7 a 3,4) 
Nivel 3: Avanzado (3,5 a 4,2) 
Nivel 4: Innovadores (4,3 a 5)' metadata={'source': 'C:/Users/Asus/Downloads/Informe_Tecnico_InstrumentoFortalecePYME.pdf', 'file_path': 'C:/Users/Asus/Downloads/Informe_Tecnico_InstrumentoFortalecePYME.pdf', 'page': 7, 'total_pages': 36, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Versión 14.4.1 (Compilación 23E224) Quartz PDFContext', 'creationDate': "D:20240531180910Z00'00'", 'modDate': "D:20240531180910Z00'00'", 'trapped': '', 'start_index': 833}


In [9]:
# Retriever
## A Retriever is a component in LangChain used to fetch relevant documents in response to a query (Chatbot retrieves relevant context before generating an answer)

from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain

@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1) # k is the number of responses by question, by relevance


retriever.batch(
    [
        "¿Cuales son los objetivos principales del documento?",
        "A qué universidad hace referencia?"
    ],
)

c:\segment_anything\.venv\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
c:\segment_anything\.venv\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[[Document(id='85658039-3db2-4d0e-be4e-a452753763c2', metadata={'source': 'C:/Users/Asus/Downloads/Informe_Tecnico_InstrumentoFortalecePYME.pdf', 'file_path': 'C:/Users/Asus/Downloads/Informe_Tecnico_InstrumentoFortalecePYME.pdf', 'page': 16, 'total_pages': 36, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Versión 14.4.1 (Compilación 23E224) Quartz PDFContext', 'creationDate': "D:20240531180910Z00'00'", 'modDate': "D:20240531180910Z00'00'", 'trapped': '', 'start_index': 1680}, page_content='órdenes de compra, recepción de materiales, y facturación (cuentas por pagar). \nPersonal \nSe recomienda implementar un gestor de recursos humanos o módulo de recursos humanos en un sistema ERP. También, \ndependiendo del número de trabajadores, podría implementar un sistema de control de asistencia  \nDatos \nSe recomienda implementar un sistema de gestor de formularios para la obtención de datos y generación de información de \npr

Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=f78ca09a-2391-4eef-a902-6ea922001641,id=f78ca09a-2391-4eef-a902-6ea922001641
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=91146e9f-deef-4d5b-bb92-82a391897b4e,id=91146e9f-deef-4d5b-bb92-82a391897b4e
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=0ae0546a-ffb

In [10]:
# Other way to do the same with vector_store ready

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1}
)

retriever.batch(
    [
        "¿Cuales son los objetivos principales del documento?",
        "A qué universidad hace referencia?"
    ],
)

[[Document(id='85658039-3db2-4d0e-be4e-a452753763c2', metadata={'source': 'C:/Users/Asus/Downloads/Informe_Tecnico_InstrumentoFortalecePYME.pdf', 'file_path': 'C:/Users/Asus/Downloads/Informe_Tecnico_InstrumentoFortalecePYME.pdf', 'page': 16, 'total_pages': 36, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Versión 14.4.1 (Compilación 23E224) Quartz PDFContext', 'creationDate': "D:20240531180910Z00'00'", 'modDate': "D:20240531180910Z00'00'", 'trapped': '', 'start_index': 1680}, page_content='órdenes de compra, recepción de materiales, y facturación (cuentas por pagar). \nPersonal \nSe recomienda implementar un gestor de recursos humanos o módulo de recursos humanos en un sistema ERP. También, \ndependiendo del número de trabajadores, podría implementar un sistema de control de asistencia  \nDatos \nSe recomienda implementar un sistema de gestor de formularios para la obtención de datos y generación de información de \npr

In [11]:
# Now using the improved search, Maximum Marginal Relevance (MMR)

retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 1}
)

retriever.batch(
    [
        "¿Cuales son los objetivos principales del documento?",
        "A qué universidad hace referencia?"
    ],
)

[[Document(id='85658039-3db2-4d0e-be4e-a452753763c2', metadata={'source': 'C:/Users/Asus/Downloads/Informe_Tecnico_InstrumentoFortalecePYME.pdf', 'file_path': 'C:/Users/Asus/Downloads/Informe_Tecnico_InstrumentoFortalecePYME.pdf', 'page': 16, 'total_pages': 36, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Versión 14.4.1 (Compilación 23E224) Quartz PDFContext', 'creationDate': "D:20240531180910Z00'00'", 'modDate': "D:20240531180910Z00'00'", 'trapped': '', 'start_index': 1680}, page_content='órdenes de compra, recepción de materiales, y facturación (cuentas por pagar). \nPersonal \nSe recomienda implementar un gestor de recursos humanos o módulo de recursos humanos en un sistema ERP. También, \ndependiendo del número de trabajadores, podría implementar un sistema de control de asistencia  \nDatos \nSe recomienda implementar un sistema de gestor de formularios para la obtención de datos y generación de información de \npr

In [6]:
####### Now we will use Retrieval Augmented Generation (RAG) ########

# Get gemini llm model from google
#from langchain_google_vertexai import ChatVertexAI
#from google.cloud.aiplatform.gapic.schema.predict.instance import SafetySetting

#llm = ChatVertexAI(model="gemini-1.5-flash")

from langchain.llms import HuggingFacePipeline
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.base import BaseCallbackHandler
from transformers import pipeline
import torch

#os.environ["LANGCHAIN_HANDLER"] = "null"
#os.environ["LANGCHAIN_TRACING"] = "false"
torch.cuda.empty_cache()
callback_manager = CallbackManager([])

print(torch.cuda.is_available())

model_name = "meta-llama/Llama-2-7b-chat-hf"
#model_name = "EleutherAI/gpt-neo-1.3B"
hf_pipeline = pipeline("text-generation", model=model_name, tokenizer=model_name, device=0) # Device=0 to GPU

llm = HuggingFacePipeline(pipeline=hf_pipeline)

# (We already have the embeddings with HuggingFace and the vector store with FAISS)

True


Loading checkpoint shards: 100%|██████████| 2/2 [00:27<00:00, 13.59s/it]
Device set to use cuda:0
C:\Users\Asus\AppData\Local\Temp\ipykernel_4828\502639593.py:26: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [15]:
from langchain import hub

# Define promt for question-answering
prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

c:\segment_anything\.venv\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=972e9d94-054c-4000-9b94-ee65ccb85577,id=972e9d94-054c-4000-9b94-ee65ccb85577


In [16]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

# For a simple RAG application, just keep track of the input question, retrieved context and generated anwser
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Let´s start with a simple sequence of two steps: retrieval and generation
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docts_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docts_content})
    response = llm.invoke(messages)
    return {"anwser": response.content}

In [7]:
question = "¿Cuales son los objetivos principales del documento?"

retrieved_docs = vector_store.similarity_search(question)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
formatted_prompt = f"question: {question}, context: {docs_content}"
answer = llm.invoke(formatted_prompt)

print(answer)


c:\segment_anything\.venv\lib\site-packages\transformers\models\llama\modeling_llama.py:566: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


KeyboardInterrupt: 